# 事前準備
## ライブラリのインポート

In [10]:
from gurobipy import Model, quicksum, GRB
import pandas as pd
import pickle

## 事前に用意されたデータの読み込み
読み込んでいる`logi_data.dump`ファイルは、同ディレクトリ内にある`make_logi_data.ipynb`ファイルで作成しています。

In [11]:
with open('logi_data.dump', 'rb') as f:
    Cust = pickle.load(f)                #顧客の集合
    DC = pickle.load(f)                   #流通センターの集合
    Route = pickle.load(f)              #配送ルートの集合
    CT = pickle.load(f)                   #センタータイプの集合
    transport = pickle.load(f)       #輸送費
    delivery = pickle.load(f)          #配送費
    dc_run = pickle.load(f)           #流通センター維持費
    dc_stock = pickle.load(f)       #流通センター在庫費
    dc_ub = pickle.load(f)            #流通センター保管容量
    dc_new = pickle.load(f)         #流通センター新規契約費
    dc_cancel = pickle.load(f)     #流通センター解約費
    dc_disposal = pickle.load(f)  #流通センター在庫廃棄費
    cust_stock = pickle.load(f)   #販社在庫費
    cust_out = pickle.load(f)       #販社品切れ費

## 計画期間の決定

In [12]:
T = 8                                             #モデルの計画期の決定
Period = list(range(1, T+1))      #計画期間のリスト

## demand.csvファイルを読み込み

In [13]:
df = pd.read_csv('demand.csv', header=None)
demand = {(j,t):df[t-1][j-1] for j in Cust for t in Period}

# モデルの構築
## モデルの作成

In [14]:
model = Model()

## 変数の追加

In [15]:
X, x, y, sD, sC, z, w, o, d = {}, {}, {}, {}, {}, {}, {}, {}, {}

#期tにおける流通センターiへの輸送量X[i,t]を追加
for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="I", name=f'X[{i},{t}]')

#期tにおける流通センターiから販社jへの配送量x[i,j,t]を追加
for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="I", name=f'x[{i},{j},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを運用するかどうかの0-1変数y[i,ct,t]を追加
for i in DC:
    for ct in CT:
        y[i,ct,0] = model.addVar(vtype="B", ub=0, name=f'y[{i},{ct},{0}]')
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

#期tにおける流通センターiの在庫量sD[i,t]を追加
for i in DC:
    sD[i,0] = model.addVar(vtype="I", ub=0, name=f'sD[{i},{t}]')
    for t in Period:
        sD[i,t] = model.addVar(vtype="I", name=f'sD[{i},{t}]')
        d[i,t] = model.addVar(vtype="I", name=f'd[{i},{t}]')

#期tにおける販社jの在庫量sC[j,t]を追加
for j in Cust:
    sC[j,0] = model.addVar(vtype="I", ub=0, name=f'sC[{j},{t}]')
    for t in Period:
        sC[j,t] = model.addVar(vtype="I", name=f'sC[{j},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを新しく建てるかどうかのバイナリ変数z[i,ct,t]を追加
for i in DC:
    for ct in CT:
        for t in Period:
            z[i,ct,t] = model.addVar(vtype="B", name=f'z[{i},{ct},{t}]')

#期tにおいて地点iにセンタータイプctの流通センターを解約するかどうかのバイナリ変数w[i,ct,t]を追加
for i in DC:
    for ct in CT:
        for t in Period:
            w[i,ct,t] = model.addVar(vtype="B", name=f'w[{i},{ct},{t}]')

#期tにおける販社jの品切れ量o[j,t]を追加
for j in Cust:
    for t in Period:
        o[j,t] = model.addVar(vtype="I", name=f'o[{j},{t}]')

#期tにおける流通センターiの在庫量廃棄量d[i,t]を追加
for i in DC:
    for t in Period:
        d[i,t] = model.addVar(vtype="I", name=f'd[{i},{t}]')

        
model.update()

## 制約の追加

In [18]:
Cust_Demand_Cons, DC_Flow_Cons, DC_Running_Cons, CT_Only_Cons, DC_UB_Cons, DC_Connect_Cons = {}, {}, {}, {}, {}, {}

#期tにおける販社jの需要を満たすための需要制約
for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC) + sC[j,t-1]
            ==
            demand[j,t] + sC[j,t] - o[j,t]
        )

#期tにおける、流通センターiのフロー整合条件
for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t] + sD[i,t-1]
            ==
            quicksum(x[i,j,t] for j in Cust) + sD[i,t] + d[i,t]
        )

#流通センターに関する0-1変数y[i,ct,t]を一意に定めるための強化制約
for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t_] for t_ in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

#期tにおいて地点iで選択できる流通センターのタイプは１つまで
for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

#期tにおける、流通センターi、センタータイプctの容量上限制約
for i in DC:
    for t in Period:
        DC_UB_Cons[i,t] = model.addConstr(
            X[i,t] + sD[i,t-1]
            <=
            quicksum(dc_ub[ct] * y[i,ct,t] for ct in  CT)
        )

#新規契約と解約をしたかどうかを一意に定める制約
for i in DC:
    for ct in CT:
        for t in Period:
            DC_Connect_Cons[i,ct,t] = model.addConstr(
                y[i,ct,t] - y[i,ct,t-1]
                ==
                z[i,ct,t] - w[i,ct,t]
            )
    
model.update()

## 目的関数の設定

In [19]:
model.setObjective(
    quicksum(transport[i] * X[i,t] for i in DC for t in Period) +                               #輸送費
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +                      #配送費
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period) +     #流通センター維持費
    quicksum(dc_stock * sD[i,t] for i in DC for t in Period) +                                #流通センター在庫費
    quicksum(cust_stock * sC[j,t] for j in Cust for t in Period) +                          #販社在庫費
    quicksum(dc_new * z[i,ct,t] for i in DC for ct in CT for t in Period) +             #流通センター新規契約費 
    quicksum(dc_cancel * w[i,ct,t] for i in DC for ct in CT for t in Period) +        #流通センター解約費
    quicksum(cust_out * o[j,t] for j in Cust for t in Period) +                                #販社品切れ費
    quicksum(dc_disposal * d[i,t] for i in DC for t in Period)                                  #流通センター廃棄費
    ,GRB.MINIMIZE
)

model.update()

## 求解

In [20]:
model.optimize()

Optimize a model with 1760 rows, 2893 columns and 11176 nonzeros
Variable types: 0 continuous, 2893 integer (1375 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+00, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 354200.00000
Presolve removed 132 rows and 308 columns
Presolve time: 0.02s
Presolved: 1628 rows, 2585 columns, 10483 nonzeros
Variable types: 0 continuous, 2585 integer (1210 binary)

Root relaxation: objective 8.113753e+04, 1223 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 81137.5261    0  145 354200.000 81137.5261  77.1%     -    0s
H    0     0                    271980.40000 81137.5261  70.2%     -    0s
H    0     0                    214480.80000 81137.5261  62.2%     -    0s
H    0     0                    176880.40000 8113

# 最適解の可視化

In [94]:
#各期ごとの総費用を辞書にする
TermObj = {t:   sum(transport[i] * float(X[i,t].x) for i in DC) +
                           sum(delivery[i,j] * float(x[i,j,t].x) for i,j in Route) +                      
                           sum(dc_run[i,ct] * float(y[i,ct,t].x) for i in DC for ct in CT) +
                           sum(dc_stock * float(sD[i,t].x) for i in DC) +                                
                           sum(cust_stock * float(sC[j,t].x) for j in Cust) +  
                           sum(dc_new * float(z[i,ct,t].x) for i in DC for ct in CT) +          
                           sum(dc_cancel * float(w[i,ct,t].x) for i in DC for ct in CT) +
                           sum(cust_out * float(o[j,t].x) for j in Cust) +                             
                           sum(dc_disposal * float(d[i,t].x) for i in DC) for t in Period}

In [103]:
print('-*-*-*-*-*-*- RESULT -*-*-*-*-*-*-')
print(f'Total Cost: {model.ObjVal}')

for t in Period:
        print(f'Term {t}')
        print(f'\tObj  {t}: {TermObj[t]}')
        
        print('\tCenter')
        print('\t\tNew   : ')
        for i in DC:
            for ct in CT:
                if z[i,ct,t].x >= 1: print(f'\t\t\tplace {i}  type {ct}')
        print('\t\tUse   : ')
        for i in DC:
            for ct in CT:
                if y[i,ct,t].x >= 1: print(f'\t\t\tplace {i}  type {ct}')
        print('\t\tCancel: ')
        for i in DC:
            for ct in CT:
                if w[i,ct,t].x >= 1: print(f'\t\t\tplace {i}  type {ct}')
        print('\tTransport')
        for i in DC:
            if X[i,t].x > 1e-5:
                print(f'\t\tto Center {i}:\t{X[i,t].x}')
        print('\tDelivery')
        for j in Cust:
            for i in DC:
                if x[i,j,t].x > 1e-5:
                    print(f'\t\tto Customer {j} from Center {i}:\t{x[i,j,t].x}')
        print('\tInventories of Center')
        for i in DC:
            dStock = False
            if sD[i,t].x > 1e-05:
                dStock = True
                print(f'\t\tCenter {i}: {sD[i,t].x}')
        if not dStock:
            print('\t\tNone')
        print('\tInventories of Customer')
        for j in Cust:
            cStock = False
            if sC[j,t].x > 1e-05:
                cStock = True
                print(f'\t\tCustomer {j}:\t{sC[j,t].x}')
        if not cStock:
            print('\t\tNone')
        print('\tLost Sales')
        for j in Cust:
            Sales = False
            if o[j,t].x > 1e-05:
                Sales = True
                print(f'\t\tCustomer {j}:\t{o[j,t].x}')
        if not Sales:
            print('\t\tNone')
        print('\tDisposal')
        for i in DC:
            disposal = False
            if d[i,t].x > 1e-05:
                disposal = True
                print('\t\tCenter {i}:\t{d[i,t].x}')
        if not disposal:
            print('\t\tNone')

-*-*-*-*-*-*- RESULT -*-*-*-*-*-*-
Total Cost: 93384.8
Term 1
	Obj  1: 11780.0
	Center
		New   : 
			place 7  type 3
			place 10  type 3
		Use   : 
			place 7  type 3
			place 10  type 3
		Cancel: 
	Transport
		to Center 7:	80.0
		to Center 10:	80.0
	Delivery
		to Customer 1 from Center 7:	5.0
		to Customer 2 from Center 7:	12.0
		to Customer 3 from Center 10:	12.0
		to Customer 4 from Center 10:	8.0
		to Customer 5 from Center 10:	15.0
		to Customer 6 from Center 7:	20.0
		to Customer 7 from Center 7:	22.0
		to Customer 8 from Center 7:	16.0
		to Customer 9 from Center 7:	5.0
		to Customer 9 from Center 10:	8.0
		to Customer 10 from Center 10:	26.0
		to Customer 11 from Center 10:	11.0
	Inventories of Center
		None
	Inventories of Customer
		None
	Lost Sales
		Customer 11:	3.0
	Disposal
		None
Term 2
	Obj  2: 8116.0
	Center
		New   : 
		Use   : 
			place 7  type 3
			place 10  type 3
		Cancel: 
	Transport
		to Center 7:	80.0
		to Center 10:	80.0
	Delivery
		to Customer 1 from Center 1